In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x3a26a80
Welcome to JupyROOT 6.26/10


In [2]:
ROOT.EnableImplicitMT(6)

In [3]:
%jsroot on

In [4]:
ROOT.gSystem.Load("libedm4hep")
ROOT.gSystem.Load("libpodio")
ROOT.gSystem.Load("libFCCAnalyses")

0

In [5]:
_edm  = ROOT.edm4hep.ReconstructedParticleData()
_pod  = ROOT.podio.ObjectID()
_fcc  = ROOT.dummyLoader

In [6]:
ROOT.TH1.SetDefaultSumw2()

In [7]:
lumi = {
    "eL.pL": 1049,
    "eL.pR": 5005,
    "eR.pL": 5767,
    "eR.pR": 1049,
}

events = {
    "eL.pL": 200000,
    "eL.pR": 51371800,
    "eR.pL": 500000,
    "eR.pR": 200000,
}

evts_per_fb = {}
for pol in lumi:
    evts_per_fb[pol] = events[pol] / lumi[pol]
    
weights = {}
for pol in evts_per_fb:
    weights[pol] = evts_per_fb[pol] / max(evts_per_fb.values())

In [8]:
print(evts_per_fb)
print(weights)

{'eL.pL': 190.65776930409913, 'eL.pR': 10264.095904095904, 'eR.pL': 86.70019074041963, 'eR.pR': 190.65776930409913}
{'eL.pL': 0.01857521315910706, 'eL.pR': 1.0, 'eR.pL': 0.008446938878057616, 'eR.pR': 0.01857521315910706}


In [9]:
files = {
    "eL.pL": "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pL.I500105.0_edm4hep.root",
    "eL.pR": "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0_edm4hep.root",
    "eR.pL": "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eR.pL.I500108.0_edm4hep.root",
    "eR.pR": "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eR.pR.I500107.0_edm4hep.root",
}
#df = ROOT.RDataFrame("events", "test_4f_sw_sl_eL_pR_edm4hep.root")
#df = ROOT.RDataFrame("events", "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0_edm4hep.root")
#df = ROOT.RDataFrame("events", "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eR.pL.I500108.0_edm4hep.root")
#df = ROOT.RDataFrame("events", files)

df = {}
for pol in files:
    df[pol] = ROOT.RDataFrame("events", files[pol])

In [10]:
df["eL.pR"].Describe()

Dataframe from TChain events in file E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0_edm4hep.root

Property                Value
--------                -----
Columns in total           39
Columns from defines        0
Event loops run             0
Processing slots            6

Column                                  Type                                            Origin
------                                  ----                                            ------
EventHeader                             ROOT::VecOps::RVec<edm4hep::EventHeaderData>    Dataset
EventHeader.eventNumber                 ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.runNumber                   ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.timeStamp                   ROOT::VecOps::RVec<ULong_t>                     Dataset
EventHeader.weight                      ROOT::VecOps::RVec<Float_t>                     Dataset
MCParticles                             ROOT::V

In [11]:
%%cpp
ROOT::VecOps::RVec<edm4hep::MCParticleData> get_enu(ROOT::VecOps::RVec<edm4hep::MCParticleData> in) {
    ROOT::VecOps::RVec<edm4hep::MCParticleData> res;
    for (size_t i = 0; i <= in.size(); i++) {
        const auto& mc = in[i];
        const auto& mc_next = in[i+1];
        if (abs(mc.PDG) == 11 && abs(mc_next.PDG) == 12) {
            res.push_back(mc);
            res.push_back(mc_next);
            break;
        }
    }
    return res;
}


In [12]:
%%cpp
ROOT::VecOps::RVec<edm4hep::MCParticleData> get_qq(ROOT::VecOps::RVec<edm4hep::MCParticleData> in) {
    ROOT::VecOps::RVec<edm4hep::MCParticleData> res;
    for (size_t i = 0; i <= in.size(); i++) {
        const auto& mc = in[i];
        const auto& mc_next = in[i+1];
        if (abs(mc.PDG) <= 5 && abs(mc_next.PDG) <= 5) {
            res.push_back(mc);
            res.push_back(mc_next);
            break;
        }
    }
    return res;
}

In [13]:
def defines(df):
       df2 = (df
              .Define("enu", "get_enu(MCParticles)")
              .Define("enu_p", "FCCAnalyses::MCParticle::get_p(enu)")
              .Define("enu_theta", "FCCAnalyses::MCParticle::get_theta(enu)")
              .Define("e", "enu[0]")
              .Define("nu", "enu[1]")
              .Define("e_p", "enu_p[0]")
              .Define("nu_p", "enu_p[1]")
              .Define("e_theta", "enu_theta[0]")
              .Define("nu_theta", "enu_theta[1]")
              .Define("e_seen", "abs(cos(e_theta)) < 0.989") # no silicon hits in ild below 8.5 deg
              .Define("enu_tlv", "FCCAnalyses::MCParticle::get_tlv(enu)")
              .Define("enu_W_tlv", "enu_tlv[0] + enu_tlv[1]")
              .Define("enu_invM", "(enu_tlv[0] + enu_tlv[1]).M()")
              .Define("qq", "get_qq(MCParticles)")
              .Define("qq_tlv", "FCCAnalyses::MCParticle::get_tlv(qq)")
              .Define("qq_W_tlv", "qq_tlv[0] + qq_tlv[1]")
              .Define("qq_invM", "qq_W_tlv.M()")
              .Define("qq_W_theta", "qq_W_tlv.Theta()")
              .Define("cos_theta_W", "cos(qq_W_theta)")
              .Define("e_tlv", "enu_tlv[0]")
              .Define("e_lv", "ROOT::Math::PxPyPzEVector(e_tlv.Px(), e_tlv.Py(), e_tlv.Pz(), e_tlv.E())")
              .Define("e_px", "e_lv.Px()")
              .Define("e_py", "e_lv.Py()")
              .Define("e_pz", "e_lv.Pz()")
              .Define("enu_lv", "ROOT::Math::PxPyPzEVector(enu_W_tlv.Px(), enu_W_tlv.Py(), enu_W_tlv.Pz(), enu_W_tlv.E())")
              .Define("qq_W_lv", "ROOT::Math::PxPyPzEVector(qq_W_tlv.Px(), qq_W_tlv.Py(), qq_W_tlv.Pz(), qq_W_tlv.E())")
              .Define("WW_lv", "qq_W_lv + enu_lv")
              .Define("e_enu_cms_lv", "ROOT::Math::Boost(enu_lv.BoostToCM())(e_lv)") # boost e to cms of enu
              .Define("enu_WW_cms_lv", "ROOT::Math::Boost(WW_lv.BoostToCM())(enu_lv)")
              .Define("em_beam_lv", "ROOT::Math::PxPyPzMVector(0, 0, 125, 0.00051)")
              .Define("em_beam_enu_cms_lv", "ROOT::Math::Boost(enu_lv.BoostToCM())(em_beam_lv)")
              .Define("em_beam_WW_cms_lv", "ROOT::Math::Boost(WW_lv.BoostToCM())(em_beam_lv)")
              .Define("e_z_axis", "enu_WW_cms_lv.Vect().Unit()")
              #.Define("e_y_axis", "ROOT::Math::XYZVector(0, 0, 1).Cross(enu_WW_cms_lv.Vect()).Unit()")
              #.Define("e_y_axis", "em_beam_enu_cms_lv.Vect().Cross(enu_WW_cms_lv.Vect()).Unit()")
              .Define("e_y_axis", "em_beam_WW_cms_lv.Vect().Cross(enu_WW_cms_lv.Vect()).Unit()")
              .Define("e_x_axis", "e_y_axis.Cross(e_z_axis)")
              .Define("e_rot", "ROOT::Math::Rotation3D(e_x_axis, e_y_axis, e_z_axis)(e_enu_cms_lv.Vect())")
              #.Define("decay_theta", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), enu_lv.Vect())")
              .Define("decay_theta", "e_rot.Theta()")
              #.Define("decay_phi", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), ROOT::Math::XYZVector(0, 0, e.charge).Cross(enu_lv.Vect().Unit()))")
              #.Define("decay_phi", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), ROOT::Math::XYZVector(0, 0, e.charge))")
              #.Define("decay_phi", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), ROOT::Math::XYZVector(0, 0, 1))") # this is still not right as it takes the shorter angle? And its not really the correct x axis
              .Define("decay_phi", "e_rot.Phi()")
              .Define("decay_cos_theta", "cos(decay_theta)")
              # for debug
              .Define("e_rot_p", "e_rot.R()")
              .Define("e_rot_px", "e_rot.X()")
              .Define("e_rot_py", "e_rot.Y()")
              .Define("e_rot_pz", "e_rot.Z()")
              .Define("e_enu_cms_p", "e_enu_cms_lv.P()")
              .Define("e_enu_cms_px", "e_enu_cms_lv.Px()")
              .Define("e_enu_cms_py", "e_enu_cms_lv.Py()")
              .Define("e_enu_cms_pz", "e_enu_cms_lv.Pz()")
              .Define("e_enu_cms_phi", "e_enu_cms_lv.Phi()")
              .Define("e_enu_cms_theta", "e_enu_cms_lv.Theta()")
              .Define("e_rot_angle", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), e_rot)")
              )
       return df2

In [14]:
df2 =  {}
for pol in df:
    df2[pol] = defines(df[pol])

In [20]:
h = {}
for pol in df2:
    h[pol] = {}
    h[pol]["e_p"] = df2[pol].Histo1D(("", ";e Momentum [GeV]", 50, 0, 150), "e_p")
    h[pol]["e_p_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} Momentum [GeV]", 50, 0, 150), "e_p")
    h[pol]["e_p_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} Momentum [GeV]", 50, 0, 150), "e_p")
    h[pol]["e_px_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} px [GeV]", 50, -75, 75), "e_px")
    h[pol]["e_px_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} px [GeV]", 50, -75, 75), "e_px")
    h[pol]["e_py_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} py [GeV]", 50, -75, 75), "e_py")
    h[pol]["e_py_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} py [GeV]", 50, -75, 75), "e_py")
    h[pol]["e_pz_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} pz [GeV]", 50, -75, 75), "e_pz")
    h[pol]["e_pz_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} pz [GeV]", 50, -75, 75), "e_pz")
    h[pol]["nu_p"] = df2[pol].Histo1D(("", ";#nu Momentum [GeV]", 50, 0, 150), "nu_p")
    h[pol]["nu_p_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} #nu Momentum [GeV]", 50, 0, 150), "nu_p")
    h[pol]["nu_p_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} #nu Momentum [GeV]", 50, 0, 150), "nu_p")
    h[pol]["e_theta"] = df2[pol].Histo1D(("", ";#theta_{e} [rad]", 50, 0, 3.142), "e_theta")
    h[pol]["e_theta_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";#theta_{e^{-}} [rad]", 50, 0, 3.142), "e_theta")
    h[pol]["e_theta_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";#theta_{e^{+}} [rad]", 50, 0, 3.142), "e_theta")
    h[pol]["nu_theta"] = df2[pol].Histo1D(("", ";#theta_{#nu} [rad]", 50, 0, 3.142), "nu_theta")
    h[pol]["enu_invM"] = df2[pol].Histo1D(("", ";e#nu inv. Mass [GeV]", 250, 0, 250), "enu_invM")
    h[pol]["enu_invM_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-}#nu inv. Mass [GeV]", 250, 0, 250), "enu_invM")
    h[pol]["enu_invM_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+}#nu inv. Mass [GeV]", 250, 0, 250), "enu_invM")
    h[pol]["qq_invM"] = df2[pol].Histo1D(("", ";q#bar{q} inv. Mass [GeV]", 250, 0, 250), "qq_invM")
    h[pol]["qq_invM_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} q#bar{q} inv. Mass [GeV]", 250, 0, 250), "qq_invM")
    h[pol]["qq_invM_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{-} q#bar{q} inv. Mass [GeV]", 250, 0, 250), "qq_invM")
    h[pol]["cos_theta_Wp"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} cos #theta_{W}", 50, -1, 1), "cos_theta_W")
    h[pol]["cos_theta_Wm"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} cos #theta_{W}", 50, -1, 1), "cos_theta_W")
    h[pol]["cos_theta_W"] = df2[pol].Histo1D(("", ";cos #theta_{W}", 50, -1, 1), "cos_theta_W")
    h[pol]["decay_cos_theta_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} cos #theta_{l}^{*}", 50, -1, 1), "decay_cos_theta")
    h[pol]["decay_cos_theta_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} cos #theta_{l}^{*}", 50, -1, 1), "decay_cos_theta")
    h[pol]["decay_cos_theta"] = df2[pol].Histo1D(("", ";cos #theta_{l}^{*}", 50, -1, 1), "decay_cos_theta")
    h[pol]["decay_phi_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} #phi_{l}^{*} [rad]", 50, -3.142, 3.142), "decay_phi")
    h[pol]["decay_phi_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} #phi_{l}^{*} [rad]", 50, -3.142, 3.142), "decay_phi")
    h[pol]["decay_phi"] = df2[pol].Histo1D(("", ";#phi_{l}^{*} [rad]", 50, -3.142, 3.142), "decay_phi")
    h[pol]["e_enu_cms_phi_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} e#nu cms #phi_{l}^{*} [rad]", 50, -3.142, 3.142), "e_enu_cms_phi")
    h[pol]["e_enu_cms_phi_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} e#nu cms #phi_{l}^{*} [rad]", 50, -3.142, 3.142), "e_enu_cms_phi")
    # debug
    h[pol]["e_rot_p_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} p^{*} [GeV]", 50, 0, 150), "e_rot_p")
    h[pol]["e_rot_p_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} p^{*} [GeV]", 50, 0, 150), "e_rot_p")
    h[pol]["e_rot_angle_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} rot angle", 50, 0, 3.143), "e_rot_angle")
    h[pol]["e_rot_angle_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{-} rot angle", 50, 0, 3.143), "e_rot_angle")
    h[pol]["e_rot_px_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} px^{*} [GeV]", 50, -75, 75), "e_rot_px")
    h[pol]["e_rot_px_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} px^{*} [GeV]", 50, -75, 75), "e_rot_px")
    h[pol]["e_rot_py_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} py^{*} [GeV]", 50, -75, 75), "e_rot_py")
    h[pol]["e_rot_py_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} py^{*} [GeV]", 50, -75, 75), "e_rot_py")
    h[pol]["e_rot_pz_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} pz^{*} [GeV]", 50, -75, 75), "e_rot_pz")
    h[pol]["e_rot_pz_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} pz^{*} [GeV]", 50, -75, 75), "e_rot_pz")
    h[pol]["e_enu_cms_p_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} e#nu cms p^{*} [GeV]", 50, 0, 150), "e_enu_cms_p")
    h[pol]["e_enu_cms_p_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} e#nu cms p^{*} [GeV]", 50, 0, 150), "e_enu_cms_p")
    h[pol]["e_enu_cms_px_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} e#nu cms px^{*} [GeV]", 50, -75, 75), "e_enu_cms_px")
    h[pol]["e_enu_cms_px_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} e#nu cms px^{*} [GeV]", 50, -75, 75), "e_enu_cms_px")
    h[pol]["e_enu_cms_py_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} e#nu cms py^{*} [GeV]", 50, -75, 75), "e_enu_cms_py")
    h[pol]["e_enu_cms_py_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} e#nu cms py^{*} [GeV]", 50, -75, 75), "e_enu_cms_py")
    h[pol]["e_enu_cms_pz_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} e#nu cms pz^{*} [GeV]", 50, -75, 75), "e_enu_cms_pz")
    h[pol]["e_enu_cms_pz_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} e#nu cms pz^{*} [GeV]", 50, -75, 75), "e_enu_cms_pz")

In [21]:
#h["enu_invM_log"] = h["enu_invM"].Clone()
h_sum = {}
first_pol = list(h)[0]
for v in h[first_pol]:
    h_sum[v] = h[first_pol][v].Clone()
    h_sum[v].Scale(weights[first_pol])
for pol in list(h)[1:]:
    for v in h[pol]:
        tmp = h[pol][v].Clone()
        #tmp.Scale(weights[pol])
        h_sum[v].Add(tmp, weights[pol])

In [22]:
c = {}
fname = "plots/no_pol.pdf"
c_dummy = ROOT.TCanvas()
c_dummy.Print(f"{fname}[")
for v in h_sum:
    c[v] = ROOT.TCanvas()
    h_sum[v].SetMinimum(0)
    h_sum[v].SetTitle("no pol")
    h_sum[v].Draw("hist")
    if (v.endswith("_log")):
        c[v].SetLogy()
    c[v].Draw()
    c[v].SaveAs(f"plots/{v}_nopol.pdf")
    c[v].Print(fname)
c_dummy.Print(f"{fname}]")



Info in <TCanvas::Print>: pdf file plots/no_pol.pdf has been created
Info in <TCanvas::Print>: pdf file plots/e_p_nopol.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file plots/no_pol.pdf
Info in <TCanvas::Print>: pdf file plots/e_p_em_nopol.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file plots/no_pol.pdf
Info in <TCanvas::Print>: pdf file plots/e_p_ep_nopol.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file plots/no_pol.pdf
Info in <TCanvas::Print>: pdf file plots/e_px_em_nopol.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file plots/no_pol.pdf
Info in <TCanvas::Print>: pdf file plots/e_px_ep_nopol.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file plots/no_pol.pdf
Info in <TCanvas::Print>: pdf file plots/e_py_em_nopol.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file plots/no_pol.pdf
Info in <TCanvas::Print>:

In [19]:
c2 = {}
for pol in h:
    c2[pol] = {}
    fname = f"plots/{pol}.pdf"
    c_dummy = ROOT.TCanvas()
    c_dummy.Print(f"{fname}[")
    for v in h[pol]:
        c2[pol][v] = ROOT.TCanvas()
        h[pol][v].SetMinimum(0)
        h[pol][v].SetTitle(pol)
        h[pol][v].Draw("hist")
        if (v.endswith("_log")):
            c2[pol][v].SetLogy()
        #c2[pol][v].Draw()
        c2[pol][v].SaveAs(f"plots/{v}_{pol}.pdf")
        c2[pol][v].Print(fname)
    c_dummy.Print(f"{fname}]")



Info in <TCanvas::Print>: pdf file plots/eL.pL.pdf has been created
Info in <TCanvas::Print>: pdf file plots/e_p_eL.pL.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file plots/eL.pL.pdf
Info in <TCanvas::Print>: pdf file plots/e_p_em_eL.pL.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file plots/eL.pL.pdf
Info in <TCanvas::Print>: pdf file plots/e_p_ep_eL.pL.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file plots/eL.pL.pdf
Info in <TCanvas::Print>: pdf file plots/e_px_em_eL.pL.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file plots/eL.pL.pdf
Info in <TCanvas::Print>: pdf file plots/e_px_ep_eL.pL.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file plots/eL.pL.pdf
Info in <TCanvas::Print>: pdf file plots/e_py_em_eL.pL.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file plots/eL.pL.pdf
Info in <TCanvas::Print>: pdf fi